# Multi step model (vector output approach)

Download zipfile from https://www.dropbox.com/s/pqenrr2mcvl0hk9/GEFCom2014.zip?dl=0 and store in the data folder.

In this notebook, we demonstrate how to:
- prepare time series data for training a RNN forecasting model
- get data in the required shape for the keras API
- implement a RNN model in keras to predict the next 24 steps ahead (time *t+1* to *t+24*) in the time series. This model uses recent values of temperature and load as the model input. The model will be trained to output a vector, the elements of which are ordered predictions for future time steps.
- enable early stopping to reduce the likelihood of model overfitting
- evaluate the model on a test dataset

The data in this example is taken from the GEFCom2014 forecasting competition<sup>1</sup>. It consists of 3 years of hourly electricity load and temperature values between 2012 and 2014. The task is to forecast future values of electricity load.

<sup>1</sup>Tao Hong, Pierre Pinson, Shu Fan, Hamidreza Zareipour, Alberto Troccoli and Rob J. Hyndman, "Probabilistic energy forecasting: Global Energy Forecasting Competition 2014 and beyond", International Journal of Forecasting, vol.32, no.3, pp 896-913, July-September, 2016.

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from glob import glob
from collections import UserDict
import itertools

%matplotlib inline

pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)

In [2]:
%run -i common/load_data.py
%run -i common/mape.py
%run -i common/TimeSeriesTensor.py
%run -i common/create_evaluation_df.py

Load data into Pandas dataframe

In [3]:
if not os.path.exists(os.path.join('data', 'energy.csv')):
    %run common/extract_data.py
energy = load_data()
energy.head()

,load,temp
2012-01-01 00:00:00,"2,698.00",32.00
2012-01-01 01:00:00,"2,558.00",32.67
2012-01-01 02:00:00,"2,444.00",30.00
2012-01-01 03:00:00,"2,402.00",31.00
2012-01-01 04:00:00,"2,403.00",32.00


In [4]:
valid_start_dt = '2014-09-01 00:00:00'
test_start_dt = '2014-11-01 00:00:00'

T = 6
HORIZON = 24
N_EXPERIMENTS = 10

In [5]:
train = energy.copy()[energy.index < valid_start_dt][['load', 'temp']]

In [6]:
from sklearn.preprocessing import MinMaxScaler

y_scaler = MinMaxScaler()
y_scaler.fit(train[['load']])

X_scaler = MinMaxScaler()
train[['load', 'temp']] = X_scaler.fit_transform(train)

## Implement the RNN

In [7]:
from keras.models import Model, Sequential
from keras.layers import GRU, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop
from keras import losses, regularizers

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


fixed parameters

In [8]:
EPOCHS = 100
ALPHA = 0.01 # regularization coefficient
N_EXPERIMENTS = 2

tunable parameters

In [9]:
LATENT_DIMS = [5, 10, 15]
BATCH_SIZES = [8, 16, 32]
LEARNING_RATES = [0.01, 0.001, 0.0001]
ALL_PARAMS = [LATENT_DIMS, BATCH_SIZES, LEARNING_RATES]

In [10]:
T_VALUES = [3, 7, 14]

In [11]:
parameters = [list(enumerate(x)) for x in ALL_PARAMS]
grid = list(itertools.product(*parameters))
lengths = [len(T_VALUES)]
lengths.extend([len(x) for x in ALL_PARAMS])
mapes = np.empty(tuple(lengths))
st_errs = np.empty_like(mapes) 

In [12]:
def create_input(T):
    
    tensor_structure = {'X':(range(-T+1, 1), ['load', 'temp'])}
    train_inputs = TimeSeriesTensor(train, 'load', HORIZON, tensor_structure)
    X_train = train_inputs.dataframe.as_matrix()[:,HORIZON:]
    Y_train = train_inputs['target']
    
    # Construct validation set (keeping T hours from the training set in order to construct initial features)
    look_back_dt = dt.datetime.strptime(valid_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
    valid = energy.copy()[(energy.index >=look_back_dt) & (energy.index < test_start_dt)][['load', 'temp']]
    valid[['load', 'temp']] = X_scaler.transform(valid)
    valid_inputs = TimeSeriesTensor(valid, 'load', HORIZON, tensor_structure)
    X_valid = valid_inputs.dataframe.as_matrix()[:,HORIZON:]
    Y_valid = valid_inputs['target']
    
    # Construct test set (keeping T hours from the validation set in order to construct initial features)
    look_back_dt = dt.datetime.strptime(test_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
    test = energy.copy()[test_start_dt:][['load', 'temp']]
    test[['load', 'temp']] = X_scaler.transform(test)
    test_inputs = TimeSeriesTensor(test, 'load', HORIZON, tensor_structure)
    X_test = test_inputs.dataframe.as_matrix()[:,HORIZON:]
    
    return X_train, Y_train, X_valid, Y_valid, X_test, test_inputs

In [13]:
def get_model(LATENT_DIM, LEARNING_RATE, T, ALPHA):
    model = Sequential()
    model.add(Dense(LATENT_DIM, activation="relu", input_shape=(2*T,), \
                    kernel_regularizer=regularizers.l2(ALPHA), bias_regularizer=regularizers.l2(ALPHA)))
    model.add(Dense(HORIZON, kernel_regularizer=regularizers.l2(ALPHA), bias_regularizer=regularizers.l2(ALPHA)))
    optimizer = RMSprop(lr=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss=losses.mean_absolute_percentage_error)
    
    return model

In [14]:
for t, T_val in enumerate(T_VALUES):
    
    X_train, Y_train, X_valid, Y_valid, X_test, test_inputs = create_input(T_val)
      
    for (i,LATENT_DIM), (j,BATCH_SIZE), (k,LEARNING_RATE) in grid:
    
        mapes_param = np.empty(N_EXPERIMENTS)
        for ii in range(N_EXPERIMENTS):
    
            # Initialize the model
            model = get_model(BATCH_SIZE, LEARNING_RATE, T_val, ALPHA)
            earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
            best_val = ModelCheckpoint('model_{epoch:02d}.h5', save_best_only=True, mode='min', period=1)
    
            # Train the model
            history = model.fit(X_train, Y_train,
                                batch_size=BATCH_SIZE,
                                epochs=EPOCHS,
                                validation_data=(X_valid, Y_valid),
                                callbacks=[earlystop, best_val],
                                verbose=0)
    
            # load the model with the smallest MAPE
            best_epoch = np.argmin(np.array(history.history['val_loss']))+1
            model.load_weights("model_{:02d}.h5".format(best_epoch))
    
            predictions = model.predict(X_test)
    
            # Compute MAPE for each forecast horizon
            eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
            eval_df['APE'] = (eval_df['prediction'] - eval_df['actual']).abs() / eval_df['actual']
    
            # Compute MAPE across all predictions
            mapes_param[ii] = mape(eval_df['prediction'], eval_df['actual'])
            print('{0:.4f}'.format(mapes_param[ii]))
    
            for f in glob('model_*.h5'):
                os.remove(f)
            
        mapes[t, i,j,k] = np.mean(mapes_param)
        st_errs[t, i,j,k] = np.std(mapes_param)/np.sqrt(N_EXPERIMENTS)
    
        result = 'Mean MAPE = {0:.4f} +/- {1:.4f}'.format(np.mean(mapes_param), np.std(mapes_param)/np.sqrt(N_EXPERIMENTS))
        print(result)

0.1705
0.0886
Mean MAPE = 0.1295 +/- 0.0290
0.0886
0.0982
Mean MAPE = 0.0934 +/- 0.0034
0.0882
0.0897
Mean MAPE = 0.0889 +/- 0.0005
0.1028
0.1072
Mean MAPE = 0.1050 +/- 0.0016
0.0802
0.0970
Mean MAPE = 0.0886 +/- 0.0059
0.0886
0.0882
Mean MAPE = 0.0884 +/- 0.0002
0.1345
0.1366
Mean MAPE = 0.1356 +/- 0.0008
0.0882
0.0886
Mean MAPE = 0.0884 +/- 0.0001
0.0870
0.1008
Mean MAPE = 0.0939 +/- 0.0049
0.0895
0.0935
Mean MAPE = 0.0915 +/- 0.0014
0.0840
0.0826
Mean MAPE = 0.0833 +/- 0.0005
0.0848
0.0849
Mean MAPE = 0.0848 +/- 0.0000
0.1216
0.1255
Mean MAPE = 0.1236 +/- 0.0014
0.0797
0.0914
Mean MAPE = 0.0856 +/- 0.0041
0.0853
0.0888
Mean MAPE = 0.0871 +/- 0.0013
0.1121
0.1794
Mean MAPE = 0.1458 +/- 0.0238
0.0865
0.1018
Mean MAPE = 0.0941 +/- 0.0054
0.0999
0.0886
Mean MAPE = 0.0943 +/- 0.0040
0.0902
0.0790
Mean MAPE = 0.0846 +/- 0.0040
0.0794
0.0907
Mean MAPE = 0.0851 +/- 0.0040
0.0916
0.0833
Mean MAPE = 0.0875 +/- 0.0029
0.1217
0.1181
Mean MAPE = 0.1199 +/- 0.0013
0.0832
0.0938
Mean MAPE = 0.0885

In [15]:
mapes.shape

(3, 3, 3, 3)

In [16]:
mapes

array([[[[1.30e-001, 9.34e-002, 8.89e-002],
         [1.05e-001, 8.86e-002, 8.84e-002],
         [1.36e-001, 8.84e-002, 9.39e-002]],

        [[9.15e-002, 8.33e-002, 8.48e-002],
         [1.24e-001, 8.56e-002, 8.71e-002],
         [1.46e-001, 9.41e-002, 9.43e-002]],

        [[8.46e-002, 8.51e-002, 8.75e-002],
         [1.20e-001, 8.85e-002, 8.92e-002],
         [1.22e-001, 9.68e-002, 9.08e-002]]],


       [[[6.22e+175, 1.14e+243, 1.34e+179],
         [2.43e-154, 3.68e+180, 1.34e+161],
         [4.18e+199, 1.22e-152, 9.79e+199]],

        [[4.82e+228, 6.01e-154, 1.11e+200],
         [5.29e+199, 3.17e+180, 1.94e+227],
         [7.09e+194, 5.98e-154, 2.46e+198]],

        [[4.16e+156, 4.24e+175, 4.98e+151],
         [6.18e+223, 5.98e-154, 9.04e-315],
         [1.09e-311, 1.09e-311, 1.09e-311]]],


       [[[1.09e-311, 1.09e-311, 1.09e-311],
         [1.09e-311, 1.09e-311, 1.09e-311],
         [1.09e-311, 1.09e-311, 1.09e-311]],

        [[1.09e-311, 1.09e-311, 1.09e-311],
         [1.09

In [18]:
T_val

14